# Agent based model for Asian Carp dispersal
## Main features
1. Package that defines Asian Carp behavior and Water Body characteristics
2. Jupyter notebook to define parameters and run simulation

## Importing packages

In [3]:
import numpy as np
from matplotlib import pyplot as plt
import geopandas as gpd

# for carp behavior
from agent_based_carp import *

# for progress
from tqdm import tqdm

# for saving parameters
import json

## Defining geometries
This cell defines the geometries that are used for the water body. Currently they are defined outside of the actual water body class. However, the water body classes are specific to particular geographies so they might as well be. 

In [4]:
# setting up MS River + BC spillway
# this is a geodataframe that only contains the geometries
lakes = gpd.read_file("shapefiles/lakes/lakes.shp")
lakes = lakes.cx[:, :0]
bonnet_carre = gpd.read_file("shapefiles/bonnet_carre.geojson")
bonnet_carre['ID'] = 1
bonnet_carre = bonnet_carre[['ID', 'geometry']]
full_water = lakes.append(bonnet_carre, ignore_index=True)
full_water.crs = "EPSG:4326"
full_water = full_water.to_crs("EPSG:32615")

# setting up upstream
# shapely geometry (NOT geodataframe)
upstream = gpd.read_file("shapefiles/upstream_river.geojson")
upstream.crs = "EPSG:4326"
upstream = upstream.to_crs("EPSG:32615")
upstream = upstream.iat[0, 0]

## Setting parameters

This next cell is for general parameter setting. It contains the established parameters for carp speed and movement patterns observed in the literature per season. In the future, we should actually test these parameters in a more systematic way.

1. Spring speed: 55km/week, 0.09 m/s
2. Summer speed: 25km/week, 0.04 m/s
3. Fall speed: 90km/week, 0.14 m/s
4. Winter speed: 12.5km/week, 0.02 m/s

In [17]:
# movement type
movement_type = {"random":2, "autocorrelation": 0, "upstream":1}
upstream_flag = True

# movement speed
density = 0.11/1000 #in carp per m^3
school_size = 10
if school_size:
    density /= school_size
carp_speed = 0.09 # in meters per second
timestep_length = 1 # in hours

distance = carp_speed*(timestep_length*60*60)
print(distance)

# movement multipliers
shoreline_multiplier = 0.25
dike_multiplier = 0.0625

# number of timesteps
simulation_length = 2*24 #in hours
timesteps = int(simulation_length/timestep_length)
print(timesteps)

# spawn area
spawn_area = gpd.read_file('shapefiles/spawn_area_2km.geojson', crs="EPSG:4326").to_crs("EPSG:32615")

# dikes
dikes = gpd.read_file("shapefiles/dikes_final.geojson")
dikes.crs = "EPSG:4326"
dikes = dikes.to_crs("EPSG:32615")
dike_distance = 100

# choose which of the dikes to use
# 0. downstream, same side
# 1. downstream, opposite side
# 2. upstream same side
# 3. upstream opposite side
# None, for no dikes
# dikes = dikes.iloc[[0]]
dikes = None

# plotting
PLOT = False


324.0
48


C:\Users\ragarcia\anaconda3\envs\asian_carp\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [4]:
%matplotlib qt
# initializing the carp
asianCarp = AsianCarp(density, distance=distance, shoreline_multiplier=shoreline_multiplier, dike_multiplier=dike_multiplier, movement_type=movement_type, upstream_flag=upstream_flag)

# initializing MS River environment
print("Loading simulation environment...")
ms_dict = {'geometry':full_water['geometry'].tolist()}
ms_river = MSRiver(asianCarp, ms_dict, spawn_area=spawn_area, upstream=upstream, crs="EPSG:32615", bc_respawn=False, dikes=dikes, dike_distance=dike_distance)
print("Starting simulation")

# list the contains the number of crossings per timestep
crossings = []

if PLOT:
    fig, ax = plt.subplots()
    ms_river.plotRiver(ax, "Start")
    # Note that using time.sleep does *not* work here!
    plt.pause(5)

for t in tqdm (range (timesteps), 
               desc="Running…", 
               ascii=False, ncols=75):
    ms_river.updateAgents()
    _, crossing_count = ms_river.respawnAgents()

    crossings.append(crossing_count)
    
    if PLOT:
        ms_river.plotRiver(ax, "timestep {}".format(t+1))


Loading simulation environment...
1009


C:\Users\ragarcia\anaconda3\envs\asian_carp\lib\site-packages\geopandas\geodataframe.py:91: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  super(GeoDataFrame, self).__setattr__(attr, val)


Starting simulation


Running…: 100%|████████████████████████████| 48/48 [04:31<00:00,  5.65s/it]


## Plotting results
This plots the number of crossings per timestep. Other metrics could be plotted.

In [5]:
# plotting the number of crossings per timestep
fig, ax = plt.subplots()
ax.plot(np.arange(1, len(crossings)+1), school_size*np.cumsum(crossings))
ax.plot(np.arange(1, len(crossings)+1), [school_size*asianCarp.num_agents]*len(crossings), 'r--')
ax.set_title("Cumulative crossings per timestep")
ax.set_xlabel("Timestep")
ax.set_ylabel("Number of crossings")
plt.show()

In [6]:
fig, ax = plt.subplots()
ms_river.plotRiver(ax, "timestep {}".format(t+1))

## Exporting parameters
This is simply to make automation of these simulations easier

In [18]:
silver = {"density":1.79/1000, "speed":[0.09, 0.04, 0.14, 0.02]}
bighead = {"density":0.42/1000, "speed":[0.18, 0.08, 0.28, 0.04]}
black = {"density":0.11/1000, "speed":[0.09, 0.04, 0.14, 0.02]}

movement_spring = {"random":2, "autocorrelation":0, "upstream":1}
movement_summer = {"random":1, "autocorrelation":1, "upstream":0}
movement_fall = {"random":2, "autocorrelation":0, "upstream":1}
movement_winter = {"random":1, "autocorrelation":1, "upstream":0}

carp_dict = {"silver":silver, "bighead":bighead, "black":black, "movement":[movement_spring, movement_summer, movement_fall, movement_winter], "upstream":[True, True, False, True]}

with open("carp.json", "w") as outfile:
    json.dump(carp_dict, outfile)